In [1]:
{-# LANGUAGE TypeOperators, FlexibleContexts, TypeFamilies #-}
import Prelude ()
import Data.Manifold.TreeCover
import Data.Random
import Data.Random.Manifold
import Data.Manifold
import Data.Manifold.Web
import Data.Manifold.DifferentialEquation
import Data.LinearMap.HerMetric
import Data.VectorSpace
import Data.Semigroup
import qualified Data.Foldable as Hask
import Control.Lens

import Control.Category.Constrained.Prelude
import Control.Arrow.Constrained
:opt no-lint  -- lint gives bogus warnings with constrained-categories

From [diagrams](http://projects.haskell.org/diagrams):

In [2]:
import Diagrams.Prelude (p2, circle, (&), moveTo, opacity, fromVertices)

From [dynamic-plot](http://hackage.haskell.org/package/dynamic-plot):

In [3]:
import Graphics.Dynamic.Plot.R2

type X = ℝ
type Y = ℝ

viewRange = plot [forceXRange (-2,4), forceYRange (-1,3)]

wPlot' :: PointsWeb ℝ (Shade' ℝ) -> DynamicPlottable
wPlot' wb = plot
            [ shapePlot $ fromVertices [p2 (x₁,y₁), p2 (x₂,y₂)]
            | ((x₁,Shade' y₁ _),(x₂,Shade' y₂ _)) <- edg ]
 where edg = webEdges wb

In [4]:
μ :: Linear ℝ (DualSpace Y) (LocalLinear Y X)
μ = denseLinear $ \λ -> denseLinear $ \δy -> δy*λ

deq :: DifferentialEqn X Y
deq = constLinearDEqn μ

In [5]:
skipN :: Int -> [a] -> [a]
skipN n xs = case splitAt n xs of
    ([], []) -> []
    ([x], []) -> [x]
    (xs₀, []) -> [head xs₀, last xs₀]
    (x:_, xs') -> x : skipN n xs'

In [14]:
tf :: PointsWeb X (Shade' Y)
tf = fromWebNodes euclideanMetric
        [(x, 1|±|[exp x*2-1.99]) | x<-[0, 0.1 .. 1.2] ]

forM_ [{- iterateFilterDEqn_static deq tf
      , -}iterateFilterDEqn_adaptive euclideanMetric deq (euclideanVolGoal 0.0001) tf]
  $ \tfs -> plotWindow $
      ( plot ((1,exp 1):±[(0.1,0),(0,0.1)] :: Shade ℝ²) ) -- Euler's number as reference for x=1
    : ( plot <$> reverse (take 20 $ skipN 5 tfs) )

![filtering the exponential function as a solution to an ODE](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/ODE-solution-filter/simple-exponential.png)

In [6]:
type YZ = ℝ²

μ₂ :: Linear ℝ (DualSpace YZ) (LocalLinear YZ X)
μ₂ = denseLinear $ \(λ,и) -> denseLinear $ \(δy,δz) -> λ*δz - и*δy

deq₂ :: DifferentialEqn X YZ
deq₂ = constLinearDEqn μ₂

In [7]:
tf₂ :: PointsWeb X (Shade' YZ)
tf₂ = fromWebNodes euclideanMetric
        [(x, (1,0)|±|[(0,δ),(δ,0)]) | x<-[0, 0.02 .. 1], let δ = 0.0001 + atan (4*x) ]

tfs₂ = iterateFilterDEqn_static deq₂ tf₂
plotWindow $
    continFnPlot sin : continFnPlot cos :
  ( plot <$> [ plot [fmap fst tffacts, fmap snd tffacts]
             | tf' <- reverse (take 20 $ skipN 15 tfs₂)
             , let tffacts = fmap factoriseShade tf' ] )

GraphWindowSpecR2{lBound=-0.16666666666666674, rBound=1.1666666666666672, bBound=-0.16666666666666669, tBound=1.1666666666666667, xResolution=640, yResolution=480}

![filtering the sin and cosine function as a solution to an ODE](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/ODE-solution-filter/simple-trigonometric.png)